In [26]:
import pandas as pd
import numpy as np
import requests
import time
import telegram
from datetime import datetime
import asyncio
import ta
from dotenv import load_dotenv
import os
load_dotenv() 

True

In [27]:
BINANCE_API = "https://api.binance.com/api/v3/klines"
BYBIT_API = "https://api.bybit.com/v2/public/kline/list"

In [28]:
TELEGRAM_TOKEN = os.getenv("TELEGRAM_TOKEN")
TELEGRAM_CHAT_ID = os.getenv("TELEGRAM_CHAT_ID")
COINS = os.getenv("COINS", "BTCUSDT,ETHUSDT").split(",")

In [29]:
if not TELEGRAM_TOKEN or not TELEGRAM_CHAT_ID:
    raise ValueError("❌ TELEGRAM_TOKEN and TELEGRAM_CHAT_ID must be set in .env file!")

print(f"✅ Loaded {len(COINS)} coins: {', '.join(COINS)}")

✅ Loaded 7 coins: BTCUSDT, ETHUSDT, SOLUSDT, BNBUSDT, DUSKUSDT,  RONINUSDT,  WIFUSDT


In [30]:
bot = telegram.Bot(token=TELEGRAM_TOKEN)

In [44]:
BINANCE_API = "https://api.binance.com/api/v3/klines"
BYBIT_API = "https://api.bybit.com/v5/market/kline"

def get_ohlcv(symbol, interval='5m', limit=500):
    try:
        url = f"{BINANCE_API}?symbol={symbol}&interval={interval}&limit={limit}"
        r = requests.get(url, timeout=10)

        if r.status_code == 200:
            data = r.json()
            if isinstance(data, list) and len(data) > 0:
                df = pd.DataFrame(data, columns=[
                    'timestamp', 'open', 'high', 'low', 'close', 'volume',
                    'close_time', 'quote_volume', 'trades',
                    'taker_buy_base', 'taker_buy_quote', 'ignore'
                ])

                df[['open','high','low','close','volume']] = df[
                    ['open','high','low','close','volume']
                ].astype(float)

                df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
                return df
    except Exception as e:
        print(f"⚠ Binance error for {symbol}: {e}")
    
    try:
        bybit_interval = interval.replace("m", "")

        params = {
            "category": "linear",
            "symbol": symbol,
            "interval": bybit_interval,
            "limit": limit
        }

        r = requests.get(BYBIT_API, params=params, timeout=10)
        if r.status_code != 200:
            return None

        data = r.json()

        if data.get("retCode") != 0:
            return None

        candles = data['result']['list']
        if not candles:
            return None

        df = pd.DataFrame(candles, columns=[
            'timestamp', 'open', 'high', 'low', 'close', 'volume', 'turnover'
        ])

        df[['open','high','low','close','volume']] = df[
            ['open','high','low','close','volume']
        ].astype(float)

        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df = df.sort_values('timestamp')

        return df

    except Exception as e:
        print(f"❌ Bybit error for {symbol}: {e}")
        return None

df = get_ohlcv("DUSKUSDT")
df.head()

,timestamp,open,high,low,close,volume,close_time,quote_volume,trades,taker_buy_base,taker_buy_quote,ignore
0,2026-01-18 17:40:00,0.1658,0.1673,0.1650,0.1668,321361.0,1768758299999,53388.23860000,338,173070.00000000,28788.44160000,0
1,2026-01-18 17:45:00,0.1665,0.1688,0.1652,0.1688,367736.0,1768758599999,61505.15920000,417,217895.00000000,36532.74040000,0
2,2026-01-18 17:50:00,0.1686,0.1710,0.1670,0.1708,436247.0,1768758899999,73985.05360000,474,260850.00000000,44307.73670000,0
3,2026-01-18 17:55:00,0.1706,0.1719,0.1697,0.1719,531375.0,1768759199999,90731.51860000,470,208680.00000000,35674.99540000,0
4,2026-01-18 18:00:00,0.1719,0.1727,0.1688,0.1694,382593.0,1768759499999,65512.03840000,414,177067.00000000,30350.32320000,0


In [46]:
def calculate_indicators(df):
    try:
        df['ema9'] = ta.trend.EMAIndicator(df['close'], window=9).ema_indicator()
        df['ema20'] = ta.trend.EMAIndicator(df['close'], window=20).ema_indicator()
        df['ema50'] = ta.trend.EMAIndicator(df['close'], window=50).ema_indicator()
        df['ema200'] = ta.trend.EMAIndicator(df['close'], window=200).ema_indicator()
        df['rsi'] = ta.momentum.RSIIndicator(df['close'], window=14).rsi()
        df['atr'] = ta.volatility.AverageTrueRange(df['high'], df['low'], df['close'], window=14).average_true_range()
        return df
    except:
        return None

df = calculate_indicators(df)
df.tail()

,timestamp,open,high,low,close,volume,close_time,quote_volume,trades,taker_buy_base,taker_buy_quote,ignore,ema9,ema20,ema50,ema200,rsi,atr
495,2026-01-20 10:55:00,0.2127,0.2147,0.2114,0.2131,110668.0,1768906799999,23570.94490000,319,55057.00000000,11739.99240000,0,0.213990,0.212010,0.207442,0.211477,54.682126,0.004696
496,2026-01-20 11:00:00,0.2134,0.2184,0.2134,0.2164,792461.0,1768907099999,171252.45690000,991,346479.00000000,74773.09470000,0,0.214472,0.212428,0.207793,0.211526,59.455798,0.004739
497,2026-01-20 11:05:00,0.2166,0.2237,0.2156,0.2222,852351.0,1768907399999,188371.15400000,1222,575074.00000000,127129.50520000,0,0.216018,0.213358,0.208358,0.211633,66.195702,0.004979
498,2026-01-20 11:10:00,0.2224,0.2244,0.2188,0.2192,536654.0,1768907699999,118910.82560000,865,245117.00000000,54319.57010000,0,0.216654,0.213915,0.208783,0.211708,60.585582,0.005023
499,2026-01-20 11:15:00,0.2195,0.2195,0.2154,0.2161,281877.0,1768907999999,61318.13330000,535,121246.00000000,26396.28440000,0,0.216543,0.214123,0.209070,0.211752,55.364078,0.004957


In [47]:
def get_mtf_trends(df_5m, symbol):
    try:
        df_15m = get_ohlcv(symbol, '15m', 300)
        df_15m = calculate_indicators(df_15m)
        
        df_1h = get_ohlcv(symbol, '1h', 200)
        df_1h = calculate_indicators(df_1h)
        
        latest_5m = df_5m.iloc[-1]
        latest_15m = df_15m.iloc[-1]
        latest_1h = df_1h.iloc[-1]
        
        # Trend detection
        trend_5m_bull = (latest_5m['ema9'] > latest_5m['ema20'] and 
                         latest_5m['ema20'] > latest_5m['ema50'] and 
                         latest_5m['ema50'] > latest_5m['ema200'])
        
        trend_5m_bear = (latest_5m['ema200'] > latest_5m['ema50'] and 
                         latest_5m['ema50'] > latest_5m['ema20'] and 
                         latest_5m['ema20'] > latest_5m['ema9'])
        
        trend_15m_bull = (latest_15m['ema9'] > latest_15m['ema20'] and 
                          latest_15m['ema20'] > latest_15m['ema50'] and 
                          latest_15m['ema50'] > latest_15m['ema200'])
        
        trend_15m_bear = (latest_15m['ema200'] > latest_15m['ema50'] and 
                          latest_15m['ema50'] > latest_15m['ema20'] and 
                          latest_15m['ema20'] > latest_15m['ema9'])
        
        trend_1h_bull = (latest_1h['ema9'] > latest_1h['ema20'] and 
                         latest_1h['ema20'] > latest_1h['ema50'] and 
                         latest_1h['ema50'] > latest_1h['ema200'])
        
        trend_1h_bear = (latest_1h['ema200'] > latest_1h['ema50'] and 
                         latest_1h['ema50'] > latest_1h['ema20'] and 
                         latest_1h['ema20'] > latest_1h['ema9'])
        
        return {
            'trend_5m_bull': trend_5m_bull,
            'trend_5m_bear': trend_5m_bear,
            'trend_15m_bull': trend_15m_bull,
            'trend_15m_bear': trend_15m_bear,
            'trend_1h_bull': trend_1h_bull,
            'trend_1h_bear': trend_1h_bear,
            'latest_5m': latest_5m,
            'latest_15m': latest_15m,
            'latest_1h': latest_1h
        }
    except:
        return None

get_mtf_trends(df, "DUSKUSDT")

{'trend_5m_bull': np.False_,
 'trend_5m_bear': np.False_,
 'trend_15m_bull': np.False_,
 'trend_15m_bear': np.False_,
 'trend_1h_bull': np.False_,
 'trend_1h_bear': np.False_,
 'latest_5m': timestamp          2026-01-20 11:15:00
 open                            0.2195
 high                            0.2195
 low                             0.2154
 close                           0.2161
 volume                        281877.0
 close_time               1768907999999
 quote_volume            61318.13330000
 trades                             535
 taker_buy_base         121246.00000000
 taker_buy_quote         26396.28440000
 ignore                               0
 ema9                          0.216543
 ema20                         0.214123
 ema50                          0.20907
 ema200                        0.211752
 rsi                          55.364078
 atr                           0.004957
 Name: 499, dtype: object,
 'latest_15m': timestamp          2026-01-20 11:15:00
 open     

In [49]:
def analyze_symbol(symbol):
    try:
        df_5m = get_ohlcv(symbol, '5m', 500)
        df_5m = calculate_indicators(df_5m)
        
        mtf = get_mtf_trends(df_5m, symbol)
        latest = mtf['latest_5m']
        
        vol_ma_5m = df_5m['volume'].rolling(20).mean().iloc[-1]
        vol_strength_5m = (df_5m['volume'].iloc[-1] / vol_ma_5m - 1) * 100 if df_5m['volume'].iloc[-1] > vol_ma_5m else 0
        vol_strength_5m = min(100, vol_strength_5m)
        
        adx_1h_proxy = mtf['latest_1h']['atr'] / mtf['latest_1h']['close'] * 100 * 2
        
        bull_score = 0
        bear_score = 0
        
        if latest['ema20'] > latest['ema50']: bull_score += 1
        if latest['close'] > latest['ema50']: bull_score += 1
        if adx_1h_proxy > 22: bull_score += 1
        if latest['rsi'] > 55: bull_score += 1
        if vol_strength_5m > 50: bull_score += 1
        if mtf['trend_15m_bull']: bull_score += 1
        
        if latest['ema200'] > latest['ema50']: bear_score += 1
        if latest['close'] < latest['ema50']: bear_score += 1
        if adx_1h_proxy > 22: bear_score += 1
        if latest['rsi'] < 45: bear_score += 1
        if vol_strength_5m > 50: bear_score += 1
        if mtf['trend_15m_bear']: bear_score += 1
        
        market_regime = "TREND" if adx_1h_proxy > 30 else "RANGE" if adx_1h_proxy > 15 else "CHOP"
        
        # S/R Levels from 15m
        df_15m = get_ohlcv(symbol, '15m', 100)
        df_15m = calculate_indicators(df_15m)
        support_15m = df_15m['low'].rolling(10).min().iloc[-1] if mtf['trend_5m_bull'] else df_15m['low'].rolling(20).min().iloc[-1]
        resistance_15m = df_15m['high'].rolling(10).max().iloc[-1] if mtf['trend_5m_bear'] else df_15m['high'].rolling(20).max().iloc[-1]
        
        # 1H S/R
        df_1h = get_ohlcv(symbol, '1h', 50)
        df_1h = calculate_indicators(df_1h)
        support_1h = df_1h['low'].rolling(10).min().iloc[-1] if mtf['trend_5m_bull'] else df_1h['low'].rolling(20).min().iloc[-1]
        resistance_1h = df_1h['high'].rolling(10).max().iloc[-1] if mtf['trend_5m_bear'] else df_1h['high'].rolling(20).max().iloc[-1]
        
        # Risk Management
        atr_mult = 1.5
        atr_value = latest['atr']
        
        stop_loss = None
        profit_target = None
        rr_ratio = 0
        
        if bull_score >= 3:
            stop_loss = max(support_15m, latest['close'] - atr_value * atr_mult)
        elif bear_score >= 3:
            stop_loss = min(resistance_15m, latest['close'] + atr_value * atr_mult)
        
        if stop_loss:
            stop_distance = abs(latest['close'] - stop_loss)
            if stop_distance > 0:
                if bull_score >= 3:
                    profit_target = min(latest['close'] + stop_distance * 2, resistance_1h)
                    rr_ratio = round((profit_target - latest['close']) / stop_distance, 2)
                elif bear_score >= 3:
                    profit_target = max(latest['close'] - stop_distance * 2, support_1h)
                    rr_ratio = round((latest['close'] - profit_target) / stop_distance, 2)
        
        return {
            'symbol': symbol,
            'price': latest['close'],
            'regime': market_regime,
            'bull_score': bull_score,
            'bear_score': bear_score,
            'stop_loss': round(stop_loss, 4) if stop_loss else None,
            'profit_target': round(profit_target, 4) if profit_target else None,
            'rr_ratio': rr_ratio,
            'vol_strength': round(vol_strength_5m, 1),
            'strong_bull': bull_score >= 3 and market_regime != "CHOP",
            'strong_bear': bear_score >= 3 and market_regime != "CHOP"
        }
    except:
        return None

analyze_symbol("DUSKUSDT")

{'symbol': 'DUSKUSDT',
 'price': np.float64(0.2155),
 'regime': 'RANGE',
 'bull_score': 2,
 'bear_score': 1,
 'stop_loss': None,
 'profit_target': None,
 'rr_ratio': 0,
 'vol_strength': 0,
 'strong_bull': False,
 'strong_bear': False}

In [35]:
async def send_telegram_message(message):
   try:
        await bot.send_message(chat_id=TELEGRAM_CHAT_ID, text=message, parse_mode='HTML')
        print(f"✅ Telegram sent: {datetime.now()}")
   except Exception as e:
        print(f"❌ Telegram error: {e}")

In [36]:
def format_telegram_message(analysis):
    emoji = "🟢" if analysis['strong_bull'] else "🔴" if analysis['strong_bear'] else "⚪"
    signal = "🚀 STRONG BULL" if analysis['strong_bull'] else "🔻 STRONG BEAR" if analysis['strong_bear'] else "⏳ WAIT"
    
    stop_str = f"<b>{analysis['stop_loss']}</b>" if analysis['stop_loss'] else "—"
    target_str = f"<b>{analysis['profit_target']}</b>" if analysis['profit_target'] else "—"
    rr_str = f"<b>{analysis['rr_ratio']}:1</b>" if analysis['rr_ratio'] > 0 else "—"
    
    message = f"""
{emoji} <b>{analysis['symbol']}</b> | ${analysis['price']:.4f}

📊 <b>Dashboard:</b>
• Regime: <b>{analysis['regime']}</b>
• Bull: <b>{analysis['bull_score']}/6</b> | Bear: <b>{analysis['bear_score']}/6</b>

🎯 <b>{signal}</b>
• Stop: {stop_str}
• Target: {target_str} 
• R:R: {rr_str}

⏰ {datetime.now().strftime('%H:%M:%S IST')}
    """
    return message.strip()

In [37]:
async def main():
    print("🚀 Pro Scalping Bot")
    print("📡 Monitoring:", ', '.join(COINS))
    
    while True:
        try:
            for symbol in COINS:
                analysis = analyze_symbol(symbol)
                
                if analysis['strong_bull'] or analysis['strong_bear']:
                    message = format_telegram_message(analysis)
                    await send_telegram_message(message)
                
                print(f"📊 {symbol}: Bull={analysis['bull_score']}/6, Bear={analysis['bear_score']}/6, Regime={analysis['regime']}")
            
            print("⏳ Waiting 60 seconds...")
            
        except KeyboardInterrupt:
            print("\n🛑 Bot stopped")
            break
        except Exception as e:
            print(f"❌ Error: {e}")